In [ ]:
import pandas as pd
import numpy as np
import logging
import shared

race_type = shared.race_type()
year = shared.forecast_year()
import time
startTime = time.time()

In [ ]:
cleaned = pd.read_csv(f'Jukola-terrain/ideal-paces-{race_type}.tsv', delimiter='\t')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={"figure.figsize":(16, 9)}) 
plot = sns.scatterplot(x="leg_distance", y="ideal_pace", hue="leg", data=cleaned)



In [ ]:

ax = sns.boxplot(x="year", y="ideal_pace", data=cleaned)

In [ ]:

ax = sns.boxplot(x="leg", y="ideal_pace", data=cleaned)

In [ ]:
runs = pd.read_csv(f'data/runs_{shared.race_id_str()}.tsv', delimiter='\t')
runs

In [ ]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'p_%s' % n
    return percentile_
runs["leg"] = runs["leg_nro"]  
leg_stats = runs[['year', "leg", "pace"]].groupby(['year', "leg"]).agg(
        ["min",percentile(1) ,percentile(5) ,"median", "mean", "std", "count"])

In [ ]:
leg_stats = leg_stats.reset_index()
leg_stats.columns = [f"{pair[0]}{pair[1]}" for pair in leg_stats.columns] 
leg_stats

In [ ]:
with_runs = pd.merge(cleaned, leg_stats, how="left", on=["year", "leg"])
with_runs

In [ ]:
with_ideal = pd.merge(runs, cleaned, how="left", left_on=["year", "leg_nro"], right_on=["year", "leg"])
with_ideal["ideal_ratio"] = with_ideal["pace"] / with_ideal["ideal_pace"]
with_ideal

In [ ]:
with_ideal = with_ideal

In [ ]:

plot = sns.scatterplot(x="team_id", y="ideal_ratio", hue="year", data=with_ideal)

In [ ]:
g = sns.FacetGrid(with_ideal, col="leg_nro", row="year")
g.map(sns.histplot, "ideal_ratio", alpha=.7)
g.add_legend()
g.set(xlim=(0, 5), ylim=(0, 180))

In [ ]:
def revquantile(n):
    def revquantile(x):
        lower = x <= n
        return np.sum(lower)/len(x)
    revquantile.__name__ = 'revq_%s' % n
    return revquantile


ratio_stats = with_ideal[['year', "leg_nro", "ideal_ratio", "ideal_pace", "pace"]].groupby(['year', "leg_nro"]).agg(
        ["min",percentile(0.5),percentile(1), percentile(2),percentile(10), "median", percentile(80),revquantile(1) ,revquantile(2) ,revquantile(3) , "mean", "std", "count"])
ratio_stats.round(3)

In [ ]:
ratio_stats_flat = ratio_stats.reset_index()
ratio_stats_flat.columns = [f"{pair[0]}_{pair[1]}" for pair in ratio_stats_flat.columns] 
ratio_stats_flat.head()

In [ ]:
ax = sns.lmplot(x="ideal_ratio_p_1", y="ideal_ratio_median", robust=True, data=ratio_stats_flat, height=7, aspect=1.5)

In [ ]:
ax = sns.regplot(x="ideal_ratio_p_0.5", y="ideal_ratio_median", data=ratio_stats_flat)

In [ ]:
ax = sns.regplot(x="ideal_ratio_p_10", y="ideal_ratio_p_80", data=ratio_stats_flat)

In [ ]:
ax = sns.regplot(x="ideal_ratio_p_0.5", y="ideal_ratio_p_10", data=ratio_stats_flat)

In [ ]:
ax = sns.lmplot(x="ideal_pace_mean", y="pace_p_1", hue="leg_nro_", data=ratio_stats_flat, height=10, aspect=1.5)

In [ ]:
ax = sns.boxplot(x="year_", y="ideal_ratio_p_1", data=ratio_stats_flat)

In [ ]:
ax = sns.boxplot(x="leg_nro_", y="ideal_ratio_p_1", data=ratio_stats_flat)